In [1]:
import numpy as np
from scipy import random
import networkx as nx
import pandas as pd
import random
import collections
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
#Model constructor
class HigherOrderNamingGame():
    
    def __init__(self, N, rule):
        
        #Structure
        self.N = N
        self.nodes = range(self.N)
        
        #Time
        self.t = 0
        
        #Rule
        self.rule = rule

    def SetInitialConditions(self, beta, p, n_A, verbose=False):
        
        #Game parameters
        self.beta = beta
        self.p = p
    
        #Opinions of the nodes (vocabularies)
        self.opinions = {}
        
        #Generating committed agents
        N_p = int(self.N*self.p) #number of committed agents
        #Randomly picking N_p committed agents
        committed = random.sample(self.nodes, N_p)
        #Setting up a committed dictionary
        #self.is_committed = {n:False for n in self.nodes}
        for n in self.nodes:
            if n in committed:
                #self.is_committed[n]=True
                #Assigning opinion "A" to committed agents
                self.opinions[n]=frozenset(["A"])
        
        #Calculating the number of agents holding n_A (not committed) from the density in input
        N_A = int(self.N*n_A)
        N_B = self.N-N_A-N_p
        #Creating a list of opinions to assign
        opinions_to_assign = ['A']*N_A + ['B']*N_B
        #Shuffling them
        random.shuffle(opinions_to_assign)
        #Agents left to be matched with opinions
        noncommitted = set(self.nodes) - set(committed)
        for n, o in zip(noncommitted, opinions_to_assign):
            self.opinions[n]=set(o)
            
        if verbose: print('Setup Done.', self.N, 'nodes,', "N_A:", N_A, "N_B:", N_B, "N_p:", N_p) 
    
    def AgreeOnSimplex(self, simplex, said_word):
        
        #Updating the simplex on the agreed word 
        for n in simplex:
            try: #There are also committed minorities which have frozensets!
                self.opinions[n].clear()
                self.opinions[n].add(said_word)
            except AttributeError: #It was committed
                pass

    def ListenersLearnWord(self, listeners, said_word):
        
        #Looping through the listeners
        for listener in listeners:
            try: #Trying to learn...
                self.opinions[listener].add(said_word)
            except AttributeError: #It was committed
                pass
                
    def play_on_simplex(self, simplex):
        
        #Selecting speaker and listeners at random
        random.shuffle(simplex)
        speaker = simplex[0]
        listeners = simplex[1:]
        
        #Selecting a random word to be said
        said_word = random.choice(list(self.opinions[speaker]))
        words_of_listeners = [self.opinions[listener] for listener in listeners]
        
        #Using the rule to get the words of listeners to be used for the agreement
        if self.rule=='union': 
            words_of_listeners_by_rule = set.union(*[set(w) for w in words_of_listeners])
        elif self.rule=='intersection':    
            words_of_listeners_by_rule = set.intersection(*[set(w) for w in words_of_listeners])
        
        #Trying to agree based on the rule and the communication efficiency beta
        if (said_word in words_of_listeners_by_rule) and (random.random() <= self.beta):
            self.AgreeOnSimplex(simplex, said_word)
        else: #No agreement, but the said word is learned by the listeners
            self.ListenersLearnWord(listeners, said_word)
            
    def get_densities(self):
        single_opinion_counter = collections.Counter([list(opinions)[0] for opinions in self.opinions.values() if len(opinions)==1])
        n_Ap = single_opinion_counter["A"]/self.N
        n_B = single_opinion_counter["B"]/self.N
        n_AB = 1-n_Ap-n_B
        return n_Ap, n_B, n_AB

    def run(self, path, t_max=100, check_every=10, print_every=1):
        
        self.t_max = t_max
        
        #Opening file to save densities results
        densities_path = path + 'HONG_densities_N%i_beta%.4f_p%.4f.csv'%(self.N, self.beta, self.p)
        f = open(densities_path,'w')
        f.write('time,n_A+p,n_B,n_AB\n')
        
        while self.t <= self.t_max:
            self.t += 1
            if self.t%print_every==0: print('t=%i'%self.t)
                            
            #Playing on a random simplex obtained as a triplet from the list of nodes
            simplex = random.sample(self.nodes, 3)
            self.play_on_simplex(simplex)
                
            #Storing the values every check_every time steps:
            if self.t%check_every==0:
                n_Ap, n_B, n_AB = self.get_densities()
                line = "%i,%.3f,%.3f,%.3f\n"%(self.t, n_Ap, n_B, n_AB)
                f.write(line)
                
                #Also checking if we reached the absorbing state:
                if n_Ap==1 or n_B==1:
                    f.close()   
                    print('DONE! Reached the absorbing state.')
                    return None
                
        f.close()    
        print('DONE! Run out of time...')

## Fixed p=0

Running n_runs simulations and saving everything in different files without any aggregation

In [3]:
N = 1000

rule = 'intersection'

betas = np.linspace(0.,1.,25)
p = 0
n_A = 0.45

t_max = 1e5
check_every = 100
print_every=50000

n_runs = 50

for run_id in range(49,50):
    for beta in betas:
        print(run_id, beta)

        output_path = '../Results/Simulations/HONG_2words_HomMix/%s/fixed_p0_varbeta_run%i/'%(rule, run_id)
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        HONG = HigherOrderNamingGame(N, rule)
        HONG.SetInitialConditions(beta=beta, p=p, n_A=n_A, verbose=True)
        HONG.run(output_path, t_max, check_every, print_every)

49 0.0
Setup Done. 1000 nodes, N_A: 450 N_B: 550 N_p: 0
t=50000
t=100000
DONE! Run out of time...
49 0.041666666666666664
Setup Done. 1000 nodes, N_A: 450 N_B: 550 N_p: 0
t=50000
t=100000
DONE! Run out of time...
49 0.08333333333333333
Setup Done. 1000 nodes, N_A: 450 N_B: 550 N_p: 0
t=50000
t=100000
DONE! Run out of time...
49 0.125
Setup Done. 1000 nodes, N_A: 450 N_B: 550 N_p: 0
t=50000
t=100000
DONE! Run out of time...
49 0.16666666666666666
Setup Done. 1000 nodes, N_A: 450 N_B: 550 N_p: 0
t=50000
t=100000
DONE! Run out of time...
49 0.20833333333333331
Setup Done. 1000 nodes, N_A: 450 N_B: 550 N_p: 0
t=50000
t=100000
DONE! Run out of time...
49 0.25
Setup Done. 1000 nodes, N_A: 450 N_B: 550 N_p: 0
t=50000
t=100000
DONE! Run out of time...
49 0.29166666666666663
Setup Done. 1000 nodes, N_A: 450 N_B: 550 N_p: 0
t=50000
t=100000
DONE! Run out of time...
49 0.3333333333333333
Setup Done. 1000 nodes, N_A: 450 N_B: 550 N_p: 0
t=50000
t=100000
DONE! Run out of time...
49 0.375
Setup Done

Aggregating each run to capture stationary/absorbing states as a function of beta

In [6]:
betas = np.linspace(0.,1.,25)

N = 1000
rule = 'intersection'
p = 0
n_A = 0.45
t_max = 1e5

n_runs = 50

for run_id in range(n_runs):

    indir = '../Results/Simulations/HONG_2words_HomMix/%s/fixed_p0_varbeta_run%i/'%(rule, run_id)


    last_k_values = 200 #for average
    #Preparing output file
    f = open(indir+'summary.csv','w')
    f.write('beta,avg_n_A+p,std_n_A+p,avg_n_B,std_n_B,avg_n_AB,std_n_AB\n')

    for beta in betas:
        #Reading file
        df = pd.read_csv(indir+'HONG_densities_N%i_beta%.4f_p%.2f.csv'%(N, beta, p))

        #Checking if simulation reached an absorbing state
        if (df.tail(1)['time'].values[0]==t_max)or(df.tail(1)['n_A+p'].values[0]==0)or(df.tail(1)['n_A+p'].values[0]==1): #Yes
            n_Ap = df.tail(1)['n_A+p'].values[0]
            n_B = df.tail(1)['n_B'].values[0]
            n_AB = df.tail(1)['n_AB'].values[0]
            line = str(beta)+','+str(n_Ap)+','+'0'+','+str(n_B)+','+'0'+','+str(n_AB)+','+'0'+'\n'
            f.write(line)
        else: #Getting stationary values
            avgdf = df.tail(last_k_values).mean()
            stddf = df.tail(last_k_values).std()

            avg_n_Ap = avgdf['n_A+p'].values[0]
            avg_n_B = avgdf['n_B'].values[0]
            avg_n_AB = avgdf['n_AB'].values[0]
            std_n_Ap = stdgdf['n_A+p'].values[0]
            std_n_B = stddf['n_B'].values[0]
            std_n_AB = stddf['n_AB'].values[0]

            line = str(beta)+','+str(avg_n_Ap)+','+str(std_n_Ap)+','+str(n_B)+','+str(std_n_B)+','+str(n_AB)+','+str(std_n_AB)+'\n'
            f.write(line)

    f.close()

## Fixed $\beta$ varying p

In [6]:
N = 1000

rule = 'intersection'

beta = 0.4
ps = np.linspace(0,0.1,25)
n_A = 0

t_max = 1e5
check_every = 100
print_every=50000

n_runs = 50

for run_id in range(n_runs):
    for p in ps:
        print(run_id, p)

        output_path = '../Results/Simulations/HONG_2words_HomMix/%s/fixed_beta_varp_run%i/'%(rule, run_id)
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        HONG = HigherOrderNamingGame(N, rule)
        HONG.SetInitialConditions(beta=beta, p=p, n_A=n_A, verbose=True)
        HONG.run(output_path, t_max, check_every, print_every)

0 0.0
Setup Done. 1000 nodes, N_A: 0 N_B: 1000 N_p: 0
DONE! Reached the absorbing state.
0 0.004166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 996 N_p: 4
t=50000
t=100000
DONE! Run out of time...
0 0.008333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 992 N_p: 8
t=50000
t=100000
DONE! Run out of time...
0 0.0125
Setup Done. 1000 nodes, N_A: 0 N_B: 988 N_p: 12
t=50000
t=100000
DONE! Run out of time...
0 0.016666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 984 N_p: 16
t=50000
t=100000
DONE! Run out of time...
0 0.020833333333333332
Setup Done. 1000 nodes, N_A: 0 N_B: 980 N_p: 20
t=50000
t=100000
DONE! Run out of time...
0 0.025
Setup Done. 1000 nodes, N_A: 0 N_B: 975 N_p: 25
t=50000
t=100000
DONE! Run out of time...
0 0.029166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 971 N_p: 29
t=50000
t=100000
DONE! Run out of time...
0 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=50000
t=100000
DONE! Run out of time...
0 0.0375
Setup Done. 1000 nodes, N_A: 0

t=100000
DONE! Run out of time...
3 0.0125
Setup Done. 1000 nodes, N_A: 0 N_B: 988 N_p: 12
t=50000
t=100000
DONE! Run out of time...
3 0.016666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 984 N_p: 16
t=50000
t=100000
DONE! Run out of time...
3 0.020833333333333332
Setup Done. 1000 nodes, N_A: 0 N_B: 980 N_p: 20
t=50000
t=100000
DONE! Run out of time...
3 0.025
Setup Done. 1000 nodes, N_A: 0 N_B: 975 N_p: 25
t=50000
t=100000
DONE! Run out of time...
3 0.029166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 971 N_p: 29
t=50000
t=100000
DONE! Run out of time...
3 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=50000
t=100000
DONE! Run out of time...
3 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=50000
t=100000
DONE! Run out of time...
3 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=50000
t=100000
DONE! Run out of time...
3 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=50000
DONE! Reached the absorbing 

t=50000
t=100000
DONE! Run out of time...
6 0.025
Setup Done. 1000 nodes, N_A: 0 N_B: 975 N_p: 25
t=50000
t=100000
DONE! Run out of time...
6 0.029166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 971 N_p: 29
t=50000
t=100000
DONE! Run out of time...
6 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=50000
t=100000
DONE! Run out of time...
6 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=50000
t=100000
DONE! Run out of time...
6 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=50000
DONE! Reached the absorbing state.
6 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=50000
DONE! Reached the absorbing state.
6 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
t=50000
DONE! Reached the absorbing state.
6 0.05416666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 946 N_p: 54
t=50000
t=100000
DONE! Run out of time...
6 0.058333333333333334
Setup Done. 1000 nodes, N_A: 0 N_B: 942 N_p: 58
t=50000
DONE! Reached the ab

t=50000
t=100000
DONE! Run out of time...
9 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=50000
t=100000
DONE! Run out of time...
9 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=50000
DONE! Reached the absorbing state.
9 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=50000
t=100000
DONE! Run out of time...
9 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
DONE! Reached the absorbing state.
9 0.05416666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 946 N_p: 54
t=50000
t=100000
DONE! Run out of time...
9 0.058333333333333334
Setup Done. 1000 nodes, N_A: 0 N_B: 942 N_p: 58
t=50000
DONE! Reached the absorbing state.
9 0.0625
Setup Done. 1000 nodes, N_A: 0 N_B: 938 N_p: 62
DONE! Reached the absorbing state.
9 0.06666666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 934 N_p: 66
t=50000
DONE! Reached the absorbing state.
9 0.07083333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 930 N_p: 70
t=50000
DONE! Reached the absorbing state.

t=50000
t=100000
DONE! Run out of time...
12 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=50000
DONE! Reached the absorbing state.
12 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
DONE! Reached the absorbing state.
12 0.05416666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 946 N_p: 54
DONE! Reached the absorbing state.
12 0.058333333333333334
Setup Done. 1000 nodes, N_A: 0 N_B: 942 N_p: 58
t=50000
DONE! Reached the absorbing state.
12 0.0625
Setup Done. 1000 nodes, N_A: 0 N_B: 938 N_p: 62
t=50000
DONE! Reached the absorbing state.
12 0.06666666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 934 N_p: 66
t=50000
DONE! Reached the absorbing state.
12 0.07083333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 930 N_p: 70
DONE! Reached the absorbing state.
12 0.075
Setup Done. 1000 nodes, N_A: 0 N_B: 925 N_p: 75
DONE! Reached the absorbing state.
12 0.07916666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 921 N_p: 79
DONE! Reached the absorbing state.
12 0.08333333

...

t=50000
t=100000
DONE! Run out of time...
37 0.016666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 984 N_p: 16
t=50000
t=100000
DONE! Run out of time...
37 0.020833333333333332
Setup Done. 1000 nodes, N_A: 0 N_B: 980 N_p: 20
t=50000
t=100000
DONE! Run out of time...
37 0.025
Setup Done. 1000 nodes, N_A: 0 N_B: 975 N_p: 25
t=50000
t=100000
DONE! Run out of time...
37 0.029166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 971 N_p: 29
t=50000
t=100000
DONE! Run out of time...
37 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=50000
t=100000
DONE! Run out of time...
37 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=50000
t=100000
DONE! Run out of time...
37 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=50000
DONE! Reached the absorbing state.
37 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=50000
DONE! Reached the absorbing state.
37 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
t=50000
DONE! Reach

t=50000
t=100000
DONE! Run out of time...
40 0.025
Setup Done. 1000 nodes, N_A: 0 N_B: 975 N_p: 25
t=50000
t=100000
DONE! Run out of time...
40 0.029166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 971 N_p: 29
t=50000
t=100000
DONE! Run out of time...
40 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=50000
DONE! Reached the absorbing state.
40 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=50000
t=100000
DONE! Run out of time...
40 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=50000
DONE! Reached the absorbing state.
40 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=50000
DONE! Reached the absorbing state.
40 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
t=50000
DONE! Reached the absorbing state.
40 0.05416666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 946 N_p: 54
t=50000
DONE! Reached the absorbing state.
40 0.058333333333333334
Setup Done. 1000 nodes, N_A: 0 N_B: 942 N_p: 58
t=50000
DONE! Rea

t=50000
t=100000
DONE! Run out of time...
43 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=50000
t=100000
DONE! Run out of time...
43 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=50000
t=100000
DONE! Run out of time...
43 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=50000
DONE! Reached the absorbing state.
43 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=50000
DONE! Reached the absorbing state.
43 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
t=50000
t=100000
DONE! Run out of time...
43 0.05416666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 946 N_p: 54
t=50000
DONE! Reached the absorbing state.
43 0.058333333333333334
Setup Done. 1000 nodes, N_A: 0 N_B: 942 N_p: 58
t=50000
DONE! Reached the absorbing state.
43 0.0625
Setup Done. 1000 nodes, N_A: 0 N_B: 938 N_p: 62
t=50000
DONE! Reached the absorbing state.
43 0.06666666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 934 N_p: 66
DONE! Reached the

t=50000
t=100000
DONE! Run out of time...
46 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=50000
DONE! Reached the absorbing state.
46 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=50000
DONE! Reached the absorbing state.
46 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
t=50000
DONE! Reached the absorbing state.
46 0.05416666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 946 N_p: 54
t=50000
DONE! Reached the absorbing state.
46 0.058333333333333334
Setup Done. 1000 nodes, N_A: 0 N_B: 942 N_p: 58
t=50000
DONE! Reached the absorbing state.
46 0.0625
Setup Done. 1000 nodes, N_A: 0 N_B: 938 N_p: 62
t=50000
DONE! Reached the absorbing state.
46 0.06666666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 934 N_p: 66
t=50000
DONE! Reached the absorbing state.
46 0.07083333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 930 N_p: 70
t=50000
DONE! Reached the absorbing state.
46 0.075
Setup Done. 1000 nodes, N_A: 0 N_B: 925 N_p: 75
DONE! Reached t

t=50000
DONE! Reached the absorbing state.
49 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
t=50000
t=100000
DONE! Run out of time...
49 0.05416666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 946 N_p: 54
t=50000
DONE! Reached the absorbing state.
49 0.058333333333333334
Setup Done. 1000 nodes, N_A: 0 N_B: 942 N_p: 58
DONE! Reached the absorbing state.
49 0.0625
Setup Done. 1000 nodes, N_A: 0 N_B: 938 N_p: 62
DONE! Reached the absorbing state.
49 0.06666666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 934 N_p: 66
DONE! Reached the absorbing state.
49 0.07083333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 930 N_p: 70
t=50000
DONE! Reached the absorbing state.
49 0.075
Setup Done. 1000 nodes, N_A: 0 N_B: 925 N_p: 75
DONE! Reached the absorbing state.
49 0.07916666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 921 N_p: 79
t=50000
DONE! Reached the absorbing state.
49 0.08333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 917 N_p: 83
t=50000
DONE! Reached the absorbing state.
49 0.

In [9]:
N = 1000

rule = 'union'

beta = 0.7
ps = np.linspace(0,0.1,25)
n_A = 0

t_max = 3e5
check_every = 100
print_every=100000

n_runs = 50

for run_id in range(n_runs):
    for p in ps:
        print(run_id, p)

        output_path = '../Results/Simulations/HONG_2words_HomMix/%s/fixed_beta_varp_run%i/'%(rule, run_id)
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        HONG = HigherOrderNamingGame(N, rule)
        HONG.SetInitialConditions(beta=beta, p=p, n_A=n_A, verbose=True)
        HONG.run(output_path, t_max, check_every, print_every)

0 0.0
Setup Done. 1000 nodes, N_A: 0 N_B: 1000 N_p: 0
DONE! Reached the absorbing state.
0 0.004166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 996 N_p: 4
t=100000
t=200000
t=300000
DONE! Run out of time...
0 0.008333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 992 N_p: 8
t=100000
t=200000
t=300000
DONE! Run out of time...
0 0.0125
Setup Done. 1000 nodes, N_A: 0 N_B: 988 N_p: 12
t=100000
t=200000
t=300000
DONE! Run out of time...
0 0.016666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 984 N_p: 16
t=100000
t=200000
t=300000
DONE! Run out of time...
0 0.020833333333333332
Setup Done. 1000 nodes, N_A: 0 N_B: 980 N_p: 20
t=100000
t=200000
t=300000
DONE! Run out of time...
0 0.025
Setup Done. 1000 nodes, N_A: 0 N_B: 975 N_p: 25
t=100000
t=200000
t=300000
DONE! Run out of time...
0 0.029166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 971 N_p: 29
t=100000
t=200000
t=300000
DONE! Run out of time...
0 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=100000
t=

t=100000
t=200000
t=300000
DONE! Run out of time...
3 0.008333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 992 N_p: 8
t=100000
t=200000
t=300000
DONE! Run out of time...
3 0.0125
Setup Done. 1000 nodes, N_A: 0 N_B: 988 N_p: 12
t=100000
t=200000
t=300000
DONE! Run out of time...
3 0.016666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 984 N_p: 16
t=100000
t=200000
t=300000
DONE! Run out of time...
3 0.020833333333333332
Setup Done. 1000 nodes, N_A: 0 N_B: 980 N_p: 20
t=100000
t=200000
t=300000
DONE! Run out of time...
3 0.025
Setup Done. 1000 nodes, N_A: 0 N_B: 975 N_p: 25
t=100000
t=200000
t=300000
DONE! Run out of time...
3 0.029166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 971 N_p: 29
t=100000
t=200000
t=300000
DONE! Run out of time...
3 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=100000
t=200000
t=300000
DONE! Run out of time...
3 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=100000
t=200000
t=300000
DONE! Run out of time...
3 0.04166

t=100000
t=200000
t=300000
DONE! Run out of time...
6 0.0125
Setup Done. 1000 nodes, N_A: 0 N_B: 988 N_p: 12
t=100000
t=200000
t=300000
DONE! Run out of time...
6 0.016666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 984 N_p: 16
t=100000
t=200000
t=300000
DONE! Run out of time...
6 0.020833333333333332
Setup Done. 1000 nodes, N_A: 0 N_B: 980 N_p: 20
t=100000
t=200000
t=300000
DONE! Run out of time...
6 0.025
Setup Done. 1000 nodes, N_A: 0 N_B: 975 N_p: 25
t=100000
t=200000
t=300000
DONE! Run out of time...
6 0.029166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 971 N_p: 29
t=100000
t=200000
t=300000
DONE! Run out of time...
6 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=100000
t=200000
t=300000
DONE! Run out of time...
6 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=100000
t=200000
t=300000
DONE! Run out of time...
6 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=100000
t=200000
t=300000
DONE! Run out of time...
6 0.0458

t=100000
t=200000
t=300000
DONE! Run out of time...
9 0.016666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 984 N_p: 16
t=100000
t=200000
t=300000
DONE! Run out of time...
9 0.020833333333333332
Setup Done. 1000 nodes, N_A: 0 N_B: 980 N_p: 20
t=100000
t=200000
t=300000
DONE! Run out of time...
9 0.025
Setup Done. 1000 nodes, N_A: 0 N_B: 975 N_p: 25
t=100000
t=200000
t=300000
DONE! Run out of time...
9 0.029166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 971 N_p: 29
t=100000
t=200000
t=300000
DONE! Run out of time...
9 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=100000
t=200000
t=300000
DONE! Run out of time...
9 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=100000
t=200000
t=300000
DONE! Run out of time...
9 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=100000
t=200000
t=300000
DONE! Run out of time...
9 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=100000
t=200000
t=300000
DONE! Run out of tim

t=100000
t=200000
t=300000
DONE! Run out of time...
12 0.020833333333333332
Setup Done. 1000 nodes, N_A: 0 N_B: 980 N_p: 20
t=100000
t=200000
t=300000
DONE! Run out of time...
12 0.025
Setup Done. 1000 nodes, N_A: 0 N_B: 975 N_p: 25
t=100000
t=200000
t=300000
DONE! Run out of time...
12 0.029166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 971 N_p: 29
t=100000
t=200000
t=300000
DONE! Run out of time...
12 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=100000
t=200000
t=300000
DONE! Run out of time...
12 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=100000
t=200000
t=300000
DONE! Run out of time...
12 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=100000
t=200000
t=300000
DONE! Run out of time...
12 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=100000
t=200000
t=300000
DONE! Run out of time...
12 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
t=100000
DONE! Reached the absorbing state.
12 0.054166

...

t=100000
t=200000
t=300000
DONE! Run out of time...
36 0.029166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 971 N_p: 29
t=100000
t=200000
t=300000
DONE! Run out of time...
36 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=100000
t=200000
t=300000
DONE! Run out of time...
36 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=100000
t=200000
t=300000
DONE! Run out of time...
36 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=100000
t=200000
t=300000
DONE! Run out of time...
36 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=100000
t=200000
t=300000
DONE! Run out of time...
36 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
t=100000
DONE! Reached the absorbing state.
36 0.05416666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 946 N_p: 54
DONE! Reached the absorbing state.
36 0.058333333333333334
Setup Done. 1000 nodes, N_A: 0 N_B: 942 N_p: 58
DONE! Reached the absorbing state.
36 0.0625
Setup Done. 1000 node

t=300000
DONE! Run out of time...
39 0.029166666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 971 N_p: 29
t=100000
t=200000
t=300000
DONE! Run out of time...
39 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=100000
t=200000
t=300000
DONE! Run out of time...
39 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=100000
t=200000
t=300000
DONE! Run out of time...
39 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=100000
t=200000
t=300000
DONE! Run out of time...
39 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=100000
t=200000
t=300000
DONE! Run out of time...
39 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
t=100000
t=200000
DONE! Reached the absorbing state.
39 0.05416666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 946 N_p: 54
DONE! Reached the absorbing state.
39 0.058333333333333334
Setup Done. 1000 nodes, N_A: 0 N_B: 942 N_p: 58
DONE! Reached the absorbing state.
39 0.0625
Setup Done. 1000 nodes, N_A: 0

t=100000
t=200000
t=300000
DONE! Run out of time...
42 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=100000
t=200000
t=300000
DONE! Run out of time...
42 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=100000
t=200000
t=300000
DONE! Run out of time...
42 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=100000
t=200000
t=300000
DONE! Run out of time...
42 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=100000
t=200000
t=300000
DONE! Run out of time...
42 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
t=100000
t=200000
t=300000
DONE! Run out of time...
42 0.05416666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 946 N_p: 54
DONE! Reached the absorbing state.
42 0.058333333333333334
Setup Done. 1000 nodes, N_A: 0 N_B: 942 N_p: 58
DONE! Reached the absorbing state.
42 0.0625
Setup Done. 1000 nodes, N_A: 0 N_B: 938 N_p: 62
DONE! Reached the absorbing state.
42 0.06666666666666667
Setup Done. 1000 nodes, N_A: 0 

t=100000
t=200000
t=300000
DONE! Run out of time...
45 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=100000
t=200000
t=300000
DONE! Run out of time...
45 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=100000
t=200000
t=300000
DONE! Run out of time...
45 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=100000
t=200000
t=300000
DONE! Run out of time...
45 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=100000
t=200000
t=300000
DONE! Run out of time...
45 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
DONE! Reached the absorbing state.
45 0.05416666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 946 N_p: 54
DONE! Reached the absorbing state.
45 0.058333333333333334
Setup Done. 1000 nodes, N_A: 0 N_B: 942 N_p: 58
DONE! Reached the absorbing state.
45 0.0625
Setup Done. 1000 nodes, N_A: 0 N_B: 938 N_p: 62
DONE! Reached the absorbing state.
45 0.06666666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 934 N_p: 66


t=300000
DONE! Run out of time...
48 0.03333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 967 N_p: 33
t=100000
t=200000
t=300000
DONE! Run out of time...
48 0.0375
Setup Done. 1000 nodes, N_A: 0 N_B: 963 N_p: 37
t=100000
t=200000
t=300000
DONE! Run out of time...
48 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 959 N_p: 41
t=100000
t=200000
t=300000
DONE! Run out of time...
48 0.04583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 955 N_p: 45
t=100000
DONE! Reached the absorbing state.
48 0.05
Setup Done. 1000 nodes, N_A: 0 N_B: 950 N_p: 50
DONE! Reached the absorbing state.
48 0.05416666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 946 N_p: 54
DONE! Reached the absorbing state.
48 0.058333333333333334
Setup Done. 1000 nodes, N_A: 0 N_B: 942 N_p: 58
DONE! Reached the absorbing state.
48 0.0625
Setup Done. 1000 nodes, N_A: 0 N_B: 938 N_p: 62
DONE! Reached the absorbing state.
48 0.06666666666666667
Setup Done. 1000 nodes, N_A: 0 N_B: 934 N_p: 66
DONE! Reached the absorbin

Aggregating each run to capture stationary/absorbing states as a function of p

In [10]:
ps = np.linspace(0,0.1,25)

N = 1000
rules = ['intersection','union']
beta = 0.7
t_maxs = [1e5,3e5]

n_runs = 50

for ir, rule in enumerate(rules):
    for run_id in range(n_runs):

        indir = '../Results/Simulations/HONG_2words_HomMix/%s/fixed_beta_varp_run%i/'%(rule, run_id)


        last_k_values = 200 #for average
        #Preparing output file
        f = open(indir+'summary_beta%.4f.csv'%beta,'w')
        f.write('p,avg_n_A+p,std_n_A+p,avg_n_B,std_n_B,avg_n_AB,std_n_AB\n')

        for p in ps:
            #Reading file
            df = pd.read_csv(indir+'HONG_densities_N%i_beta%.4f_p%.4f.csv'%(N, beta, p))

            #Checking if simulation reached an absorbing state
            if (df.tail(1)['time'].values[0]==t_maxs[ir])or(df.tail(1)['n_A+p'].values[0]==0)or(df.tail(1)['n_A+p'].values[0]==1): #Yes
                n_Ap = df.tail(1)['n_A+p'].values[0]
                n_B = df.tail(1)['n_B'].values[0]
                n_AB = df.tail(1)['n_AB'].values[0]
                line = str(p)+','+str(n_Ap)+','+'0'+','+str(n_B)+','+'0'+','+str(n_AB)+','+'0'+'\n'
                f.write(line)
            else: #Getting stationary values
                avgdf = df.tail(last_k_values).mean()
                stddf = df.tail(last_k_values).std()

                avg_n_Ap = avgdf['n_A+p']#.values[0]
                avg_n_B = avgdf['n_B']#.values[0]
                avg_n_AB = avgdf['n_AB']#.values[0]
                std_n_Ap = stddf['n_A+p']#.values[0]
                std_n_B = stddf['n_B']#.values[0]
                std_n_AB = stddf['n_AB']#.values[0]

                line = str(p)+','+str(avg_n_Ap)+','+str(std_n_Ap)+','+str(n_B)+','+str(std_n_B)+','+str(n_AB)+','+str(std_n_AB)+'\n'
                f.write(line)

        f.close()

## Fixed $p$ varying $\beta$

In [3]:
N = 1000

rule = 'intersection'

p = 0.04 #[0.02, 0.08, 0.16]
betas= np.linspace(0,1,25)
n_A = 0

t_max = 1e5
check_every = 100
print_every=50000

n_runs = 50

for run_id in range(n_runs):
    for beta in betas:
        print(run_id, beta)

        output_path = '../Results/Simulations/HONG_2words_HomMix/%s/fixed_p_varbeta_run%i/'%(rule, run_id)
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        HONG = HigherOrderNamingGame(N, rule)
        HONG.SetInitialConditions(beta=beta, p=p, n_A=n_A, verbose=True)
        HONG.run(output_path, t_max, check_every, print_every)

0 0.0
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.08333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.125
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.16666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.20833333333333331
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.25
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.29166666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.375
Setup Done. 1000 nodes, N_A: 

t=50000
t=100000
DONE! Run out of time...
3 0.125
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.16666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.20833333333333331
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.25
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.29166666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
3 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of ti

t=50000
t=100000
DONE! Run out of time...
6 0.25
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
6 0.29166666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
6 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
6 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
6 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
6 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
6 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
6 0.5416666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
6 0.5833333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time..

t=50000
t=100000
DONE! Run out of time...
9 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
9 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
9 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
9 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
9 0.5416666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
9 0.5833333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
9 0.625
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
9 0.6666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
9 0.7083333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time..

t=50000
t=100000
DONE! Run out of time...
12 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
12 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
12 0.5416666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
12 0.5833333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
12 0.625
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
12 0.6666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
12 0.7083333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
12 0.75
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
12 0.7916666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of 

...

t=50000
t=100000
DONE! Run out of time...
37 0.08333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.125
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.16666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.20833333333333331
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.25
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.29166666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run 

t=50000
t=100000
DONE! Run out of time...
40 0.16666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.20833333333333331
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.25
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.29166666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out

t=50000
t=100000
DONE! Run out of time...
43 0.25
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
43 0.29166666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
43 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
43 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
43 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
43 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
43 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
43 0.5416666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
43 0.5833333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out 

t=50000
t=100000
DONE! Run out of time...
46 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
46 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
46 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
46 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
46 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
46 0.5416666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
46 0.5833333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
46 0.625
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
46 0.6666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out 

t=50000
DONE! Reached the absorbing state.
49 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
49 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
49 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
49 0.5416666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
49 0.5833333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
49 0.625
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
49 0.6666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
49 0.7083333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
49 0.75
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out 

In [4]:
N = 1000

rule = 'union'

p = 0.04 #[0.02, 0.08, 0.16]
betas= np.linspace(0,1,25)
n_A = 0

t_max = 1e5
check_every = 100
print_every=50000

n_runs = 50

for run_id in range(n_runs):
    for beta in betas:
        print(run_id, beta)

        output_path = '../Results/Simulations/HONG_2words_HomMix/%s/fixed_p_varbeta_run%i/'%(rule, run_id)
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        HONG = HigherOrderNamingGame(N, rule)
        HONG.SetInitialConditions(beta=beta, p=p, n_A=n_A, verbose=True)
        HONG.run(output_path, t_max, check_every, print_every)

0 0.0
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.041666666666666664
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.08333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.125
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.16666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.20833333333333331
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.25
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.29166666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
0 0.375
Setup Done. 1000 nodes, N_A: 

t=50000
t=100000
DONE! Run out of time...
3 0.125
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.16666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.20833333333333331
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.25
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.29166666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
3 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
3 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing 

t=50000
t=100000
DONE! Run out of time...
6 0.25
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
6 0.29166666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
6 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
6 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
6 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
6 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
DONE! Reached the absorbing state.
6 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
6 0.5416666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
DONE! Reached the absorbing state.
6 0.5833333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
6 0.625
Se

t=50000
t=100000
DONE! Run out of time...
9 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
9 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
9 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
9 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
9 0.5416666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
9 0.5833333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
DONE! Reached the absorbing state.
9 0.625
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
9 0.6666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
9 0.7083333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
9 

t=50000
t=100000
DONE! Run out of time...
12 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
12 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
12 0.5416666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
12 0.5833333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
DONE! Reached the absorbing state.
12 0.625
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
12 0.6666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
12 0.7083333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
12 0.75
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
12 0.7916666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time

...

t=50000
t=100000
DONE! Run out of time...
37 0.08333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.125
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.16666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.20833333333333331
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.25
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.29166666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
37 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the a

t=50000
t=100000
DONE! Run out of time...
40 0.16666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.20833333333333331
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.25
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.29166666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
40 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
40 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
DONE! Reached the absorbing s

t=50000
t=100000
DONE! Run out of time...
43 0.25
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
43 0.29166666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
43 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
43 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
43 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
43 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
43 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
43 0.5416666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
DONE! Reached the absorbing state.
43 0.5833333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing 

t=50000
t=100000
DONE! Run out of time...
46 0.3333333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
46 0.375
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
46 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
46 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
DONE! Reached the absorbing state.
46 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
46 0.5416666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
DONE! Reached the absorbing state.
46 0.5833333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
DONE! Reached the absorbing state.
46 0.625
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
46 0.6666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
46 0.7083

t=50000
t=100000
DONE! Run out of time...
49 0.41666666666666663
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
49 0.4583333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
DONE! Reached the absorbing state.
49 0.5
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
49 0.5416666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
DONE! Reached the absorbing state.
49 0.5833333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
49 0.625
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
49 0.6666666666666666
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
49 0.7083333333333333
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of time...
49 0.75
Setup Done. 1000 nodes, N_A: 0 N_B: 960 N_p: 40
t=50000
t=100000
DONE! Run out of tim

Aggregating each run to capture stationary/absorbing states as a function of $\beta$

In [3]:
betas= np.linspace(0,1,25)

N = 1000
rules = ['intersection','union']
p = 0.04 #[0.02, 0.08, 0.16]

n_runs = 50
t_max = 1e5

for rule in rules:
    for run_id in range(n_runs):

        indir = '../Results/Simulations/HONG_2words_HomMix/%s/fixed_p_varbeta_run%i/'%(rule, run_id)


        last_k_values = 200 #for average
        #Preparing output file
        f = open(indir+'summary_p%.4f.csv'%p,'w')
        f.write('beta,avg_n_A+p,std_n_A+p,avg_n_B,std_n_B,avg_n_AB,std_n_AB\n')

        for beta in betas:
            #Reading file
            df = pd.read_csv(indir+'HONG_densities_N%i_beta%.4f_p%.4f.csv'%(N, beta, p))

            #Checking if simulation reached an absorbing state
            if (df.tail(1)['time'].values[0]==t_max)or(df.tail(1)['n_A+p'].values[0]==0)or(df.tail(1)['n_A+p'].values[0]==1): #Yes
                n_Ap = df.tail(1)['n_A+p'].values[0]
                n_B = df.tail(1)['n_B'].values[0]
                n_AB = df.tail(1)['n_AB'].values[0]
                line = str(beta)+','+str(n_Ap)+','+'0'+','+str(n_B)+','+'0'+','+str(n_AB)+','+'0'+'\n'
                f.write(line)
            else: #Getting stationary values
                avgdf = df.tail(last_k_values).mean()
                stddf = df.tail(last_k_values).std()

                avg_n_Ap = avgdf['n_A+p']#.values[0]
                avg_n_B = avgdf['n_B']#.values[0]
                avg_n_AB = avgdf['n_AB']#.values[0]
                std_n_Ap = stddf['n_A+p']#.values[0]
                std_n_B = stddf['n_B']#.values[0]
                std_n_AB = stddf['n_AB']#.values[0]

                line = str(beta)+','+str(avg_n_Ap)+','+str(std_n_Ap)+','+str(n_B)+','+str(std_n_B)+','+str(n_AB)+','+str(std_n_AB)+'\n'
                f.write(line)

        f.close()